# Simulating genomic data in a 'plant breeding' context

In [1]:
using XSim, DelimitedFiles, Distributions

### Defining the genome and reading a real marker catalogue

In [2]:
mutRate    = 0.0
numChr     = 2
nLoci      = 6096
chrLength  = [1.62;1.41]
numLoci    = [3339;2757]
mutationRate = 0.0
myData = readdlm("markerCatalogue4JuliaChrom1-2", ' ', Any, '\n', header=false)
mp1 = Float64.(myData[1,1:numLoci[1]])
mp2 = Float64.(myData[2,1:numLoci[2]])
mapPos = [mp1, mp2]

genefreq1   = fill(0.5,numLoci[1])
genefreq2 = fill(0.5,numLoci[2])
geneFreq = [genefreq1, genefreq2]

idx = rand(numLoci[1]).>0.995  # you want 0.5% to be QTL, i.e about 17 QTL
qtlIndex1 = collect(1:numLoci[1])[idx]
idx = rand(numLoci[2]).>0.995  # you want 0.5%% to be QTL, i.e about 14 QTL
qtlIndex2 = collect(1:numLoci[2])[idx]
qtlIndex = [qtlIndex1, qtlIndex2]
line = 0
numQTL = 0

for i in qtlIndex
    line +=1
    println("Number of QTL on chromosome $line: ", length(i))
    numQTL += 1
end

qtlEffect1 = randn(length(qtlIndex1))/sqrt(0.5*numQTL)
qtlEffect2 = randn(length(qtlIndex2))/sqrt(0.5*numQTL)
qtlEffect = [qtlEffect1, qtlEffect2];

Number of QTL on chromosome 1: 18
Number of QTL on chromosome 2: 11


### Building the genome
Passing  arrays for chrLength, numLoci and arrays of arrays for mapPos, geneFreq, qtlIndex, qtlEffect
i.e. using the most flexible version of build_genome

In [3]:
build_genome(numChr,chrLength,numLoci,geneFreq, mapPos, qtlIndex, qtlEffect, mutationRate)

### Sampling founder individuals from a file with 1326 genotyped individuals
Note that the genotype data for two chromosomes on the file is phased. You may have to use FImpute or some similar software for phasing. You may read less individuals form the data file than it contains, but not more.

In [4]:
popSizeFounder = 1326
basePop = sampleFounders(popSizeFounder, "reformattedMarkerDataChrom1-2");

Sampling 1326 animals into base population.


Splitting the founder individuals into tow cohorts of equal size

In [5]:
basePopMales

UndefVarError: UndefVarError: basePopMales not defined

In [6]:
basePopMales = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
basePopMales.animalCohort = basePop.animalCohort[1:663];

In [7]:
basePopFemales = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
basePopFemales.animalCohort = basePop.animalCohort[664:end];

Although not used here, animals and cohorts belong to breeds, and the breed coposition can be set for cohorts and is inferered thereafter from the mating strategy

In [8]:
XSim.setBreedComp(basePop,[1.0])
XSim.setBreedComp(basePopMales,[1.0])
XSim.setBreedComp(basePopFemales,[1.0]);

Removing output files form a previous run, then creating 5'000 DH lines from an initial crossing of the two base populations

In [9]:
outputFileName = "JuliaPlants"
run(`\rm -f $outputFileName.ped`)
run(`\rm -f $outputFileName.phe`)
run(`\rm -f $outputFileName.brc`)
run(`\rm -f $outputFileName.gen`)

numOffspring = 5000
DHBaseLines1 = XSim.sampleDHLines(basePopMales, basePopFemales, numOffspring)
DHBaseLines2 = XSim.sampleDHLines(basePopMales, basePopFemales, numOffspring);

Lets look at the mean and variance of the genotypic values of our 5000 DH-lines in DHBaseline1 and DHBaseline2.  

In [10]:
var(getOurGenVals(DHBaseLines1))

10.546984038692457

In [11]:
var(getOurGenVals(DHBaseLines2))

10.635990251420086

In [12]:
mean(getOurGenVals(DHBaseLines1))

9.33250667292257

In [13]:
mean(getOurGenVals(DHBaseLines2))

9.332454511145675

Let us specify  the residual variance to simulate phenotypic values, note that getOurPhenVals() is actually setting all individuals phenotypic values in the cohort that is passed as an argument.

In [14]:
resVar = 5
XSim.setResidualVariance(resVar)

5

In [15]:
var(getOurPhenVals(DHBaseLines1,resVar))

15.332143285659404

In [16]:
var(getOurPhenVals(DHBaseLines2,resVar))

15.970021195774667

In [17]:
outputFileName

"JuliaPlants"

Now lets output the data to the files. Note that outputPedigree() is not only outputting the pedigree, but also the phenotypes and gentoypes, look at the files that got generated!

In [18]:
outputPedigree(DHBaseLines1,outputFileName);
outputPedigree(DHBaseLines2,outputFileName);

Sampling each 2500 male and female parents from DHBaseline1 at random and producing one offspring each. Sampling of parents is done with replacement. Note that sampleRan() is sampling 50% female and 50% male offspring, we are concatenating them into a single cohort. Note further, that we are using the same population to sample male and female parents from, so sex is ignored and we might even end up cloning an individual. Furthermore, after this step of random mating, our DH-Lines are not double haploids anymore. With the parameter numGen that is passed as an argument to sampleRan(), we are determining the number of generations of random mating. The variable 'gen' is used to store the actual numer of generation the simualtion currently is at.

In [60]:
DHLine1_H1 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H1m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H1f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numOffspring = 2
numGen = 1
gen = 1
output = true
DHLine1_H1m, DHLine1_H1f, gen = sampleRan(numOffspring, numGen, DHBaseLines1, DHBaseLines1, fileName=outputFileName)
DHLine1_H1 = concatCohorts(DHLine1_H1m, DHLine1_H1f);

Generation     2: sampling     1 males and     1 females


In [59]:
DHLine1_H1

XSim.Cohort(XSim.Animal[Animal(XSim.Chromosome[Chromosome([1, 1, 1, 1, 1, 1, 1, 0, 0, 1  …  1, 0, 1, 1, 0, 0, 1, 1, 0, 0], [1932, 302, 1932], [0.0, 0.72077, 0.857245]), Chromosome([0, 0, 1, 0, 0, 1, 1, 1, 0, 1  …  0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1931, 301], [0.0, 0.198435])], XSim.Chromosome[Chromosome([1, 1, 1, 1, 0, 0, 1, 0, 0, 1  …  0, 1, 1, 1, 1, 1, 1, 0, 1, 0], [471, 1567], [0.0, 1.22784]), Chromosome([0, 0, 0, 0, 0, 0, 0, 1, 1, 1  …  1, 0, 0, 1, 0, 1, 0, 1, 0, 1], [1567, 471], [0.0, 1.11788])], [1.0], 15525, 4263, 4939, 8.83906, 9.76703, -9999.0), Animal(XSim.Chromosome[Chromosome([1, 0, 1, 1, 0, 0, 1, 0, 0, 0  …  0, 0, 0, 1, 1, 1, 1, 1, 0, 1], [1587, 994, 1587], [0.0, 0.165055, 0.61528]), Chromosome([0, 0, 1, 1, 0, 0, 1, 1, 0, 1  …  1, 0, 0, 1, 0, 1, 0, 1, 0, 1], [993, 1587], [0.0, 0.566497])], XSim.Chromosome[Chromosome([2, 0, 0, 0, 1, 1, 0, 0, 1, 1  …  1, 1, 0, 0, 1, 1, 1, 0, 1, 0], [2366, 994], [0.0, 0.840472]), Chromosome([1, 0, 1, 1, 0, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 0, 1, 0

In [54]:
## mating individuals for a given pedigree ###??
mutable struct PedNode
    ind::Int64
    sire::Int64
    dam::Int64
end

Now we are repeating the same random mating for DHBaseline2

In [36]:
#DHLine2_H1 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
#DHLine2_H1m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
#DHLine2_H1f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
#numOffspring = 5000
#numGen = 1
#gen = 1
#output = true

true

In [21]:
#DHLine2_H1m, DHLine2_H1f, gen = sampleRan(numOffspring, numGen, DHBaseLines2, DHBaseLines2, fileName=outputFileName)
#DHLine2_H1 = concatCohorts(DHLine2_H1m, DHLine2_H1f);

Generation     2: sampling   250 males and   250 females


We know now how to generate double haploid lines and how to do random mating. Next let's do phenotypic selection. As before we are declaring the male and female cohorts to hold the offspring that are going to be generated. We will be generation 1000 offspring based on 100 phenotypically best male and female parents. Again we will be selecting the phenotypically best 100 parents from the same population that was generated in the previous step (DHLine1_H1).

In [22]:
DHLine1_H2 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H2m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H2f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numMaleParents = 1
numFemaleParents = 1
numOffspring = 2500
numGen = 10  ### results in RILs????
DHLine1_H2m, DHLine1_H2f, gen = sampleSel(numOffspring, numMaleParents, numFemaleParents, numGen, 
                                    DHLine1_H1, DHLine1_H1, XSim.common.varRes, gen=gen,
                                    fileName=outputFileName, direction=1);
DHLine1_H2 = concatCohorts(DHLine1_H2m, DHLine1_H2f);

Generation     3: sampling   125 males and   125 females


As before, we are applying mass selection also to the other population in the same way as was done above for DHLine_H1 now for DHLine_H2

In [23]:
DHLine2_H2 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine2_H2m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine2_H2f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numMaleParents = 100
numFemaleParents = 100
numOffspring = 2500
numGen = 1

1

In [24]:
DHLine2_H2m, DHLine2_H2f, gen = sampleSel(numOffspring, numMaleParents, numFemaleParents, numGen, 
                                    DHLine2_H1, DHLine2_H1, XSim.common.varRes, gen=gen,
                                    fileName=outputFileName, direction=1);
DHLine2_H2 = concatCohorts(DHLine2_H2m, DHLine2_H2f);

Generation     4: sampling   125 males and   125 females


After random mating and phentoypic selection, let us apply selection based on BLUP, i.e. breeding values. For this purpose, we need to read the pedigree and phenotpyic files that were produced sampleRan() and sampleSel() as well as outputPedigree() above. First, let's read the phenotypic data into a dataframe.

In [25]:
using DataFrames
# BLUP selection
phenofile = outputFileName*".phe"
colNames = [:Animal;:y;:bv]
#dfPhen = CSV.read(phenofile,delim = ' ',header=false,names=colNames)
dfPhen = readtable(phenofile, separator=' ', header=false)
names!(dfPhen, colNames)
dfPhen[:1] = string.(dfPhen[:1])
dfPhen

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = ip:0x0
└ @ Core :-1


,Animal,y,bv
,String,Float64⍰,Float64⍰
1,1327,7.923,12.629
2,1328,9.19,7.119
3,1329,8.038,10.179
4,1330,5.205,5.302
5,1331,7.667,8.302
6,1332,12.318,10.999
7,1333,14.449,15.53
8,1334,9.767,11.176
9,1335,5.391,6.741


┌ Warning: In the future eachcol will have names argument set to false by default
│   caller = getmaxwidths(::DataFrame, ::UnitRange{Int64}, ::UnitRange{Int64}, ::Symbol) at show.jl:105
└ @ DataFrames /opt/julia/packages/DataFrames/5Rg4Y/src/abstractdataframe/show.jl:105


Now, we read the pedigree in to a dataframe, reformat it, write is out again such that it can finally be read by the get_pedigree() method. This should be fixed in future releases....

In [26]:
pedfile   = outputFileName*".ped"
dfPed = readtable(pedfile,separator = ' ', header = false)

,x1,x2,x3
,Int64⍰,Int64⍰,Int64⍰
1,1327,182,1003
2,1328,60,1316
3,1329,439,1075
4,1330,188,722
5,1331,497,742
6,1332,59,1025
7,1333,319,1263
8,1334,365,709
9,1335,547,745


In [27]:
using CSV
pedfile = outputFileName*".pedReformatted"
CSV.write(pedfile, dfPed, delim=';',header=["ind","sire" , "dam"])

"JuliaPlants.pedReformatted"

In [28]:
ped = XSim.get_pedigree(pedfile,separator=';');

coding pedigree... 100%|████████████████████████████████| Time: 0:00:00


Finished!


calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


After having read phenotypes and genotypes, we can now set the genetic variance that shall be used in BLUP, define the model for analysis, setup the mixed model equations and solve them. 

In [29]:
varGen = 5
mme = XSim.build_model("y = intercept + Animal",resVar)
XSim.set_random(mme,"Animal",ped,varGen)
out = XSim.solve(mme,dfPhen,solver="GaussSeidel",printout_frequency=40)

40 0.019937347914262903
80 0.0011204398588182374
120 6.296679586625604e-5
160 3.5386258011326756e-6


14830×2 Array{Any,2}:
 "1:intercept : intercept"   9.37551  
 "1:Animal : 258"           -1.16067  
 "1:Animal : 1251"          -2.46195  
 "1:Animal : 5734"          -1.41621  
 "1:Animal : 304"            2.06991  
 "1:Animal : 1246"          -3.02278  
 "1:Animal : 10467"         -0.0161252
 "1:Animal : 295"           -3.39674  
 "1:Animal : 762"            0.138801 
 "1:Animal : 8169"          -2.67048  
 "1:Animal : 599"           -2.73161  
 "1:Animal : 228"            5.74799  
 "1:Animal : 483"            1.14446  
 ⋮                                    
 "1:Animal : 3564"           1.13221  
 "1:Animal : 10745"          0.40854  
 "1:Animal : 9377"           0.173526 
 "1:Animal : 4355"          -0.247073 
 "1:Animal : 13621"          0.10327  
 "1:Animal : 14760"          4.2851   
 "1:Animal : 7319"           4.55923  
 "1:Animal : 6583"           1.70867  
 "1:Animal : 3276"          -1.54132  
 "1:Animal : 12795"         -1.80661  
 "1:Animal : 13939"         -2.24068  
 "1

Now all our indiviudals have estimated genotypic values, with the method putEBV() we are transferring them to the indiviudals that we would like to select to become the parents of the next generation. In our case, these would be the cohorts DHLine1_H2 and DHLine2_H2

In [30]:
# transfer BLUP-EBV to animals
XSim.putEBV(DHLine1_H2,ped,mme,out)
XSim.putEBV(DHLine2_H2,ped,mme,out)

In [31]:
DHLine1_H2_sorted = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine2_H2_sorted  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine1_H3  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine2_H3  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
direction = 1.0
numMaleParents = 50
numFemaleParents = 50
numOffspring = 1250
numGen = 1

1

After having declared empty cohorts for the offspring of the next generation, we are creating a y vector with the animals EBVs in the direction we want to select on (e.g. if low values arefavourable, direction should be -1.0). Then EBVs are sorted and the best number of parents are selected to be passed as parents to sampleChildren()

In [32]:
println("Generation $gen : sampling children by BLUP")
y = direction*[animal.ebv for animal in DHLine1_H2.animalCohort]
DHLine1_H2_sorted.animalCohort = DHLine1_H2.animalCohort[sortperm(y)][(end-numMaleParents+1):end]
y = direction*[animal.ebv for animal in DHLine2_H2.animalCohort]
DHLine2_H2_sorted.animalCohort = DHLine2_H2.animalCohort[sortperm(y)][(end-numFemaleParents+1):end]
DHLine1_H3 = XSim.sampleChildren(DHLine1_H2_sorted,DHLine1_H2_sorted,numOffspring)
DHLine2_H3 = XSim.sampleChildren(DHLine2_H2_sorted,DHLine2_H2_sorted,numOffspring)
outputPedigree(DHLine1_H3,outputFileName)
outputPedigree(DHLine2_H3,outputFileName)
gen+=1

Generation 4 : sampling children by BLUP


5

Finally, we are producing each 250 double haploid lines again, from the offspring of the BLUP selected parents 

In [33]:
println("Producing DHLines again....")
DHLine1_4  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine2_4  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))

numOffspring = 1250
DHLine1_4 = XSim.sampleDHLines(DHLine1_H3, DHLine1_H3, numOffspring)
DHLine2_4 = XSim.sampleDHLines(DHLine2_H3, DHLine2_H3, numOffspring);
     


Producing DHLines again....


In [34]:
outputPedigree(DHLine1_4,outputFileName);
outputPedigree(DHLine2_4,outputFileName);

The final step is crossing the two DHLines again to produce hybrids....

In [35]:
println("Producing hybrids now....")
H1 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
H1m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
H1f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numOffspring = 1250
numGen = 1
gen = 1
output = true
H1m, H1f, gen = sampleRan(numOffspring, numGen, DHLine1_4, DHLine2_4, fileName=outputFileName)
H1 = concatCohorts(DHLine2_H1m, DHLine2_H1f);


Producing hybrids now....
Generation     2: sampling    62 males and    62 females


Of course the 'breeding program' outlined here is not reflecting a real situation but rahter shows you the tools XSim has to do simulations. You should be able to modify the above code to better reflect a real situation. Also you may modify XSim's methodsto better reflect the situation you need to cope with. Note that all data simulated is written to the files with the name <outputFileName> that you declared above. so You should be able to look at selection response, genetic trend across generation etc.